# 1NN против RandomForest
## Programming assignment

В этом задании будет использоваться датасет digits из sklearn.datasets. Оставьте последние 25% объектов для контроля качества, разделив X и y на X_train, y_train и X_test, y_test.

In [27]:
from sklearn import datasets, metrics, ensemble
from scipy.spatial.distance import euclidean


def write_answer_to_file(answer, filename):
    with open(filename, 'w') as f_out:
        f_out.write(str(answer))
        
digits = datasets.load_digits()
amount_of_data = digits.data.shape[0]

X_train, X_test = digits['data'][:int(amount_of_data * 0.75), :], digits['data'][int(amount_of_data * 0.75):, :]
y_train, y_test = digits['target'][:int(amount_of_data * 0.75)], digits['target'][int(amount_of_data * 0.75):]

Целью задания будет реализовать самый простой метрический классификатор — метод ближайшего соседа, а также сравнить качество работы реализованного вами 1NN с RandomForestClassifier из sklearn на 1000 деревьях.

**Задание 1.** Реализуйте самостоятельно метод одного ближайшего соседа с евклидовой метрикой для задачи классификации. Можно не извлекать корень из суммы квадратов отклонений, т.к. корень — монотонное преобразование и не влияет на результат работы алгоритма.

Никакой дополнительной работы с признаками в этом задании делать не нужно — мы еще успеем этим заняться в других курсах. Ваша реализация может быть устроена следующим образом: можно для каждого классифицируемого объекта составлять список пар (расстояние до точки из обучающей выборки, метка класса в этой точке), затем сортировать этот список (по умолчанию сортировка будет сначала по первому элементу пары, затем по второму), а затем брать первый элемент (с наименьшим расстоянием).

Сортировка массива длиной N требует порядка N log N сравнений (строже говоря, она работает за O(N log N)). Подумайте, как можно легко улучшить получившееся время работы. Кроме простого способа найти ближайший объект всего за N сравнений, можно попробовать придумать, как разбить пространство признаков на части и сделать структуру данных, которая позволит быстро искать соседей каждой точки. За выбор метода поиска ближайших соседей в KNeighborsClassifier из sklearn отвечает параметр algorithm — если у вас уже есть некоторый бэкграунд в алгоритмах и структурах данных, вам может быть интересно познакомиться со структурами данных ball tree и kd tree.

Доля ошибок, допускаемых 1NN на тестовой выборке, — ответ в задании 1.

In [17]:
def one_nearest_neighbors(X_test, X_train, y_train):
    predictions = []
    for i in range(X_test.shape[0]):
        distances = []
        for j in range(X_train.shape[0]):
            distances.append((euclidean(X_test[i], X_train[j]), y_train[j]))
        predictions.append(sorted(distances)[0][1])
    return predictions

In [25]:
y_pred = one_nearest_neighbors(X_test, X_train, y_train)
answer_1 = metrics.zero_one_loss(y_test, y_pred)  # 1 - accuracy_score(y_test, y_pred)  
write_answer_to_file(answer_1, '1NN_vs_RandomForest_answer_1.txt')

**Задание 2.** Теперь обучите на обучающей выборке RandomForestClassifier(n_estimators=1000) из sklearn. Сделайте прогнозы на тестовой выборке и оцените долю ошибок классификации на ней. Эта доля — ответ в задании 2. Обратите внимание на то, как соотносится качество работы случайного леса с качеством работы, пожалуй, одного из самых простых методов — 1NN. Такое различие — особенность данного датасета, но нужно всегда помнить, что такая ситуация тоже может иметь место, и не забывать про простые методы.

In [28]:
rf_clf = ensemble.RandomForestClassifier(n_estimators=1000)
rf_clf.fit(X_train, y_train)

answer_2 = metrics.zero_one_loss(y_test, rf_clf.predict(X_test))
write_answer_to_file(answer_2, '1NN_vs_RandomForest_answer_2.txt')

In [37]:
print('One nearest neighbors quality: {:.2f}'.format(1 - answer_1))
print('Random Forest with 1000 trees quality: {:.2f}'.format(1 - answer_2))

One nearest neighbors quality: 0.96
Random Forest with 1000 trees quality: 0.93
